In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Libs
import logging
import pandas as pd
from collections import OrderedDict
import pickle
import torch

# Own Modules
from source_torch.mlca.mlca import mlca_mechanism
from source_torch.mlca.mlca_setup import set_value_model_parameters
#from source.mlca.mlca import mlca_mechanism
#from source.mlca.mlca_setup import set_value_model_parameters

import datetime
import os
from source_torch.util import save_result, load_result
import numpy as np

MLCA NN Class imported
MLCA NN_MIP Class imported
WDP Class imported
MLCA_Economies imported
MLCA function imported


In [3]:
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% | 12% |
|  1 | 12% |  8% |
|  2 | 24% | 32% |
|  3 | 16% | 32% |
|  4 | 12% |  8% |
|  5 | 12% |  8% |
|  6 | 19% | 16% |
|  7 |  0% |  0% |


In [4]:
torch.cuda.set_device(1)
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device

In [11]:
iterations = 1
seed = 0
while iterations > 0:
    x = datetime.datetime.now()
    m = x.month
    d = x.day
    h = x.hour
    mi = x.minute

    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    # log debug to console
    logging.basicConfig(level=logging.DEBUG, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')
    #logging.basicConfig(level=logging.WARNING, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')
    #logging.basicConfig(level=logging.INFO, format='%(asctime)s:               %(message)s', datefmt='%H:%M:%S')    
    
    configdict_mlca = OrderedDict([('SATS_domain_name','GSVM'),
                        ('SATS_auction_instance_seed', seed),
                        ('bidders',0),
                        ('items',0),
                        ('bidder_ids',0),
                        ('Qinit', 30),
                        ('Qmax', 50),#50
                        ('Qround',0),
                        ('Starter','mlca_extra'),
                        ('epochs' , 512),#512
                        ('batch_size' , 32),
                        ('regularization_type' , 'l1_l2'),
                        ('layer_N' , [10, 10]),
                        ('layer_R' ,[32, 32] ),
                        ('layer_L' , [16, 16]),
                        ('NN_parameters',[]),
                        ('bigM',6000),
                        ('warm_start',False),
                        ('MIP_parameters',[]),
                        ('scaler',None),
                        ('init_bids_and_fitted_scaler',[None,None]),
                        ('return_allocation',True),
                        ('return_payments',True),
                        ('lra',0.01),
                        ('regularization',1e-5),
                        ('calc_efficiency_per_iteration',True),
                        ('active_learning_algorithm', 'unif'),
                        ('presampled_n', 20),
                        ('presampled_algorithm', 'unif')])

    configdict_mlca = set_value_model_parameters(configdict_mlca)

    res = mlca_mechanism(configdict = configdict_mlca)
    seed += 1

    if res == 'UNIQUENESS CHECK FAILED see logfile':
        # add one more iteration to the loop
        iterations += 1
        #continue

    iterations -= 1

    dirname = './experiments/MLCA/Torch/results/' + configdict_mlca['SATS_domain_name'] +'_'+ configdict_mlca['active_learning_algorithm']
    result_dir = './experiments/MLCA/Torch/results/' + configdict_mlca['SATS_domain_name'] +'_' + configdict_mlca['active_learning_algorithm'] +'/' +str(m)+'_'+str(d) + '_' + str(h) + '_' + str(mi) + '_' + str(configdict_mlca['SATS_auction_instance_seed']-1)
    os.makedirs(dirname,exist_ok=True)
    save_result(result_dir, res)

18:28:48:               START MLCA:
18:28:48:               -----------------------------------------------
18:28:48:               Model: GSVM
18:28:48:               Seed SATS Instance: 0
18:28:48:               Qinit: 30
18:28:48:               Qmax: 50
18:28:48:               Qround: 7

18:28:48:               Instantiate SATS Instance
18:28:48:               Set NN parameters
18:28:48:               Set MIP parameters
18:28:48:               INITIALIZE BIDS
18:28:48:               -----------------------------------------------

18:28:48:               Set up initial Bids for: Bidder_0
18:28:48:               Sampling uniformly at random 30 bundle-value pairs from bidder 0
18:28:48:               Set up initial Bids for: Bidder_1
18:28:48:               Sampling uniformly at random 30 bundle-value pairs from bidder 1
18:28:48:               Set up initial Bids for: Bidder_2
18:28:48:               Sampling uniformly at random 30 bundle-value pairs from bidder 2
18:28:48:          

MLCA NN_MIP Class imported

------------------------ SATS parameters ------------------------
Value Model:  GSVM
Number of Bidders:  7
Number of BidderTypes:  2
Number of Items:  18
Scaler:  None

------------------------ DNN  parameters ------------------------
Epochs: 512
Batch Size: 32
Regularization: l1_l2

Bidder_0:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2

Bidder_1:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2

Bidder_2:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2

Bidder_3:
regularization: 1e-05
learning_rate: 0.01
architecture: [32, 32]
dropout: True
dropout_prob: 0.05
epochs: 512
batch_size: 32
regularization_type: l1_l2

Bidder_4:
regularization: 1e-05
lea

18:28:48:               Status  : integer optimal solution
18:28:48:               Time    : 0 sec
18:28:48:               Problem : MILP
18:28:48:               Rel. Gap: 0.0 %
18:28:48:               N. Iter : 19
18:28:48:               Hit Lim.: False
18:28:48:               Objective Value: 289.8641939691543
18:28:50:               INFO
18:28:50:               -----------------------------------------------
18:28:50:               Seed Auction Instance: 0
18:28:50:               Iteration of MLCA: 1
18:28:50:               Number of Elicited Bids:
18:28:50:               Bidder_0: 30
18:28:50:               Bidder_1: 30
18:28:50:               Bidder_2: 30
18:28:50:               Bidder_3: 30
18:28:50:               Bidder_4: 30
18:28:50:               Bidder_5: 30
18:28:50:               Bidder_6: 30
18:28:50:               Qinit: 30 | Qround: 7 | Qmax: 50
18:28:50:               Efficiency given elicited bids from iteration 0-0: 0.5737198084935572

18:28:50:               Current

First time in optimization step.
18
18


18:29:02:               Tighten bounds with IA for Bidder_0
18:29:02:               Tighten bounds with IA for Bidder_2
18:29:02:               Tighten bounds with IA for Bidder_3
18:29:02:               Tighten bounds with IA for Bidder_4
18:29:02:               Tighten bounds with IA for Bidder_5
18:29:02:               Tighten bounds with IA for Bidder_6
18:29:02:               Sorted active bidders in MIP: ['Bidder_0', 'Bidder_2', 'Bidder_3', 'Bidder_4', 'Bidder_5', 'Bidder_6']
18:29:02:               Mip initialized
18:29:02:               Solving MIP
18:29:02:               Attempt no: 1
18:29:02:               Mip time Limit of 1800
18:29:02:               Mip relative gap 0.001
18:29:02:               Mip integrality tol 1e-08


...

19:05:20:               Tighten bounds with IA for Bidder_0
19:05:20:               Tighten bounds with IA for Bidder_1
19:05:20:               Tighten bounds with IA for Bidder_2
19:05:20:               Tighten bounds with IA for Bidder_3
19:05:20:               Tighten bounds with IA for Bidder_4
19:05:20:               Tighten bounds with IA for Bidder_5
19:05:20:               Tighten bounds with IA for Bidder_6
19:05:20:               Sorted active bidders in MIP: ['Bidder_0', 'Bidder_1', 'Bidder_2', 'Bidder_3', 'Bidder_4', 'Bidder_5', 'Bidder_6']
19:05:20:               Adding bidder specific constraints
19:05:21:               Mip initialized
19:05:21:               Solving MIP
19:05:21:               Attempt no: 1
19:05:21:               Mip time Limit of 1800
19:05:21:               Mip relative gap 0.001
19:05:21:               Mip integrality tol 1e-08


Time limit is 1800


19:06:10:               -----------------------------------------------
19:06:10:               Status  : integer optimal, tolerance
19:06:10:               Time    : 49 sec
19:06:10:               Problem : MILP
19:06:10:               Rel. Gap: 0.00095 %
19:06:10:               N. Iter : 773221
19:06:10:               Hit Lim.: False
19:06:10:               SET ARGMAX ALLOCATION ONLY BIDDER SPECIFIC for Bidder_6
19:06:10:               Bidder_0:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] | [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
19:06:10:               Bidder_1:  [0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.] | [0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
19:06:10:               Bidder_2:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] | [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
19:06:10:               Bidder_3:  [0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0.] | [0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.

Time for MIP run: 49 sec
Found objective 487.32085094281246


19:06:10:               Status  : integer optimal solution
19:06:10:               Time    : 0 sec
19:06:10:               Problem : MILP
19:06:10:               Rel. Gap: 0.0 %
19:06:10:               N. Iter : 35
19:06:10:               Hit Lim.: False
19:06:10:               Objective Value: 505.23651036254824
19:06:10:               ALLOCATION
19:06:10:               ---------------------------------------------
19:06:10:               Calculate MLCA allocation: Main Economy
19:06:10:               Active bidders: ['Bidder_0', 'Bidder_1', 'Bidder_2', 'Bidder_3', 'Bidder_4', 'Bidder_5', 'Bidder_6']
19:06:10:               MLCA allocation in Main Economy:
19:06:10:               Bidder_0 {'good_ids': [], 'value': 0}
19:06:10:               Bidder_1 {'good_ids': [2, 3, 4, 5, 13], 'value': 126.06866770098435}
19:06:10:               Bidder_2 {'good_ids': [], 'value': 0}
19:06:10:               Bidder_3 {'good_ids': [6, 7, 8, 9, 14, 15, 16], 'value': 226.88376094852987}
19:06:10:       

In [ ]:
results = load_result(result_dir)

./experiments/MLCA/Torch/results/GSVM_galo/8_17_12_52_-1 is loaded 


In [ ]:
# check if all vectors are unique in  results[0]['Elicited Bids']['Bidder_1'][0]
results['MLCA Efficiency']

0.9801468713664441